In [1]:
import os
import re
import pandas as pd
import importlib

## Extracción

In [3]:
import FuncionesLectura as fl
importlib.reload(fl)

df_corpus=fl.leerCorpus(hacerLimpieza=True)
print(df_corpus.shape)
df_corpus.sample(5)

Antes de limpiar: (6471, 9)
Después de eliminar español: (6102, 9)
Después de eliminar vacíos: (6101, 9)
Después de limpiar duplicados: (5623, 9)
(5621, 9)


,id,speaker,transcription,text,morpheme_break,pos,gloss_es,free_translation,file
5810,DICCIONARIOISKONAWA7_1054,Dictionary,neheh,None,None,n.,hoacin o shansho,"animal del tamaño de una gallina, de color mar...",DICCIONARIOISKONAWA7.txt
2913,JP-cuento.animal-2013_060,jose,a retei bokanke retei ka,a retei bokanke retei ...,a rete -i bo -kan -ke rete...,dem v. -suf. v.ambi -suj. -suf. v....,ese matar(SK) -SUJ>S:SIM ir:PL -PLU -PERF(SK...,Se fueron a matar.,JP-cuento.animal-2013.txt
4970,DICCIONARIOISKONAWA7_214,Dictionary,betahi,None,None,v.tran.,cubri,cubrir algo con una manta o u objeto similar.,DICCIONARIOISKONAWA7.txt
793,JC-historias.iskonawa-2013_085,Juanita,mara chikishke chicho mara chikishke,mara chikishke chicho mara ...,ma =ra chikish -ke chicho ma =r...,adv =clit. adj. -suf. n. adv =clit....,ya =EVID1(SK) ocioso -PERF(SK) hermana.mayor...,mi hermana ya está cansada,JC-historias.iskonawa-2013.txt
5520,DICCIONARIOISKONAWA7_764,Dictionary,koman ewa,None,None,n.,perdiz o perdiz,"Ave de tamaño grande, color cenizo oscuro y pi...",DICCIONARIOISKONAWA7.txt


In [4]:
#Se separa el corpus en dos partes, uno normal y otro del diccionario

df_diccionario=df_corpus[df_corpus["speaker"]=="diccionario"]
df_corpus=df_corpus[df_corpus["speaker"]!="diccionario"]

print(df_corpus.shape)
print(df_diccionario.shape)

df_corpus.to_json('corpus.json', orient='records', lines=True)
df_diccionario.to_json('diccionario.json', orient='records', lines=True)

## Prueba de lectura

In [5]:
df_corpus_leido = pd.read_json('corpus.json', lines=True)
df_diccionario_leido = pd.read_json('diccionario.json', lines=True)
df_corpus_leido.sample(5)

,id,speaker,transcription,text,morpheme_break,pos,gloss_es,free_translation,file
1835,JC.NC.IC-paucar.del.mani-2013_100,Juanita,"mia betsosi iki ""ashpaketere""",mia betsosi iki ashpaketere,mia be- tsos -i ik -i as...,pron pref.- v. -suf. v.intr. -suf....,2SG.ABS(SK) cara- lanzar -NOMLZ copula -IMPF...,"""te voy a disparar en la frente, vuelan las pl...",JC.NC.IC-paucar.del.mani-2013.txt
1807,JC.NC.IC-paucar.del.mani-2013_072,Juanita,karestimari epá,karestimari epá,ka -res -ti =ma =ri epá,v.intr -suf. -suf. =clit. =clit n.,ir -solamente -NOMLZ =NEG =COP.INT papá.VOC,"""vamos a caminar no más, papá""",JC.NC.IC-paucar.del.mani-2013.txt
900,JC-vida.juanita-2013_157,Juanita,ainoah menoribike,ainoah menoribike,ain =no =ah meno =ribi -ke,adv. =clit. =suf. n. =clit. -suf.,ahí(MIX) =LOC =S quemar(SK) =también -PER...,de ahí también lo quemaron,JC-vida.juanita-2013.txt
1589,JC.NC.IC-los.mezquinos-2013_114,Juanita,ato animaribo kenamatimari,ato animaribo kenamatimari,ato animari =bo kena -ma -ti ma =ri,pron. n. =clit v. -suf. -suf. adv =clit,3PL animal =PLU llamar -CAUS -NOMLZ ya =C...,¿haremos llamar a los animales?,JC.NC.IC-los.mezquinos-2013.txt
3651,NC-narracion.vida-2013.eaf_a32,Nelita,mahonka apati apati atipayamai,mahonka apati apati atipayamai,None,None,None,"cuando yo cocinaba, no podía bajar (la olla).",NC-narracion.vida-2013.eaf


## Validaciones

In [6]:
#Verificar que las columnas id,	speaker,	transcription, free_translation, file no tengan valores nulos
assert df_corpus_leido.id.isnull().sum() == 0
assert df_corpus_leido.speaker.isnull().sum() == 0  
assert df_corpus_leido.transcription.isnull().sum() == 0
assert df_corpus_leido.free_translation.isnull().sum() == 0
assert df_corpus_leido.file.isnull().sum() == 0

print('No hay valores nulos en el corpus')

No hay valores nulos en el corpus
